In [3]:
import pyprind
import pandas as pd
import os
pbar = pyprind.ProgBar(50000)
labels = {'pos':1,'neg':0}
df = pd.DataFrame()
for s in ('test','train'):
    for l in ('pos','neg'):
        path = 'C:/Users/SKJ/Documents/GitHub/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path,file),'rb') as infile:
                txt = infile.read()
            df = df.append([[txt,labels[l]]],ignore_index=True)
        pbar.update()
df.columns = ['review','sentiment']

In [4]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('C:/Users/SKJ/Documents/GitHub/aclImdb/movie_data.csv',index=False)

NameError: name 'df' is not defined

In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/SKJ/Documents/GitHub/aclImdb/movie_data.csv')
df.head(3)

,review,sentiment
0,"b'In 1974, the teenager Martha Moxley (Maggie ...",1
1,"b""OK... so... I really like Kris Kristofferson...",0
2,"b'***SPOILER*** Do not read this, if you think...",0


In [2]:
#词袋模型
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)
print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [3]:
#清洗文本数据
df.loc[0,'review'][-50:]

"s seven.<br /><br />Title (Brazil): Not Available'"

In [4]:
#正则表达式库
import re
def preprocessor(text):
    text = re.sub('<[^>]*>',' ',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(D|P)',text)
    text = re.sub('[\W]+',' ',text.lower()) + ' '.join(emoticons).replace('-',' ')
    return text

In [5]:
preprocessor(df.loc[0,'review'][-50:])

's seven title brazil not available '

In [6]:
preprocessor("</a>This :) is  :( a test :-)!")

' this is a test :) : )'

In [7]:
df['review'] = df['review'].apply(preprocessor)

In [8]:
#标记文档
def tokenizer(text):
    return text.split()

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [9]:
#停用词移除
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running \
and runs a lot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SKJ\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['runner', 'like', 'run', 'run', 'lot']

In [10]:
#训练用于文档分类的逻辑回归模型
X_train = df.loc[:25000,'review'].values
y_train = df.loc[:25000,'sentiment'].values
X_test = df.loc[25000:,'review'].values
y_test = df.loc[25000:,'sentiment'].values

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop,None],
               'vect__tokenizer': [tokenizer,tokenizer_porter],
               'clf__penalty': ['l1','l2'],
               'clf__C': [1.0,10.0,100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop,None],
               'vect__tokenizer': [tokenizer,tokenizer_porter],
               'clf__penalty': ['l1','l2'],
               'clf__C': [1.0,10.0,100.0]}]

lr_tfidf = Pipeline([('vect',tfidf),
                    ('clf',LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf,param_grid,
                           scoring = 'accuracy',
                           cv=5,verbose=1,n_jobs=-1)
gs_lr_tfidf.fit(X_train,y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
print("Best parameter set: %s " % gs_lr_tfidf.best_params_)
print("CV Accuracy: %.3f" % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print("Test Accuracy: %.3f" % clf.score(X_test,y_test))